In [1]:
%run ../00_default_options.ipynb

In [2]:
from robotehr.pipelines import training
from robotehr.models.cohort import Cohort, OnsetDataFrame
from robotehr.models.data import FeaturePipeline
from robotehr.pipelines.supporters.preprocessing import DataLoader

In [3]:
cohort = Cohort.load(id=1)

In [5]:
onset_dataframe = OnsetDataFrame.load(id=17)

In [6]:
feature_pipeline = FeaturePipeline.load(id=19)

In [7]:
import morpher
from morpher.jobs import *

Using TensorFlow backend.


In [8]:
agg_func_regex = 'count__'

def prepare_data_function(data, target):
    import morpher
    from morpher.jobs import Impute, Scale
    from sklearn.preprocessing import OneHotEncoder
    from enum import Enum
    
    # remove unused features
    del data['medical_record_number']
    del data['mother_account_number']
    del data['date_of_birth']
    del data['month_of_birth']
    del data['patient_ethnic_group']
    del data['religion']
    del data['address_zip']
    del data['deceased_indicator']
    del data['marital_status_code']
    
    class RaceType(str, Enum):
        AFRICAN = 'African'
        AMERICAN_BLACK = 'Black or African-American'
        AMERICAN_NATIVE = 'Native American'
        ASIAN = 'Asian'
        ASIAN_PACIFIC = 'Asian Pacific'
        ASIAN_INDIAN = 'Asian Indian'
        ASIAN_CHINESE = 'Asian Chinese'
        HISPANIC = 'Hispanic or Latino'
        OTHER = 'Other'
        WHITE = 'White'
    
    RACE_MAPPING = {
        RaceType.AFRICAN: [
            'Cape Verdian',
            'Congolese',
            'Eritrean',
            'Ethiopian',
            'Gabonian',
            'Ghanaian',
            'Guinean',
            'Ivory Coastian',
            'Kenyan',
            'Liberian',
            'Madagascar',
            'Malian',
            'Nigerian',
            'Other: East African',
            'Other: North African',
            'Other: South African',
            'Other: West African',
            'Senegalese',
            'Sierra Leonean',
            'Somalian',
            'Sudanese',
            'Tanzanian',
            'Togolese',
            'Ugandan',
            'Zimbabwean'
        ],
        RaceType.AMERICAN_BLACK: [
            'African American (Black)',
            'African-American',
            'Black Or African-American',
            'Black or African - American',
        ],
        RaceType.AMERICAN_NATIVE: [
            'American (Indian/Alaskan)',
            'Native American'
        ],
        RaceType.ASIAN: [
            'Asian',
            'Bangladeshi',
            'Bhutanese',
            'Burmese',
            'Cambodian',
            'Hmong',
            'Indonesian',
            'Japanese',
            'Korean',
            'Laotian',
            'Malaysian',
            'Maldivian',
            'Nepalese',
            'Okinawan',
            'Pakistani',
            'Singaporean',
            'Taiwanese',
            'Thai',
            'Vietnamese',
            'Yapese'
        ],
        RaceType.ASIAN_PACIFIC: [
            'Asian (Pacific Islander)',
            'Carolinian',
            'Chamorro',
            'Chuukese',
            'Fijian',
            'Filipino',
            'Guamanian',
            'Guamanian Or Chamorro',
            'Guamanian or Chamorro',
            'Iwo Jiman',
            'Kiribati',
            'Kosraean',
            'Mariana Islander',
            'Marshallese',
            'Melanesian',
            'Micronesian',
            'Native Hawaiian',
            'New Hebrides',
            'Other Pacific Islander',
            'Pacific Islander',
            'Palauan',
            'Pohnpeian',
            'Polynesian',
            'Saipanese',
            'Samoan',
            'Papua New Guinean',
            'Tahitian',
            'Tokelauan',
            'Tongan'
        ],
        RaceType.ASIAN_INDIAN: [
            'Asian Indian',
            'Sri Lankan',
            'Sri lankan',
            'West Indian'
        ],
        RaceType.ASIAN_CHINESE: [
            'Chinese',
        ],
        RaceType.HISPANIC: [
            'Barbadian',
            'Dominica Islander',
            'Grenadian',
            'Haitian',
            'Hispanic/Latino',
            'Jamaican',
            'St Vincentian',
            'Trinidadian'
        ],
        RaceType.OTHER: [
            '',
            'Aa',
            'Ab',
            'Af',
            'Ag',
            'Ak',
            'Al',
            'Ap',
            'Ar',
            'Av',
            'Ay',
            'B',
            'B1',
            'B2',
            'B3',
            'B4',
            'B5',
            'B6',
            'B7',
            'B8',
            'B9',
            'Ba',
            'Bb',
            'Bc',
            'Bd',
            'Be',
            'Bf',
            'Bg',
            'Bh',
            'Bj',
            'Bk',
            'Bm',
            'Bn',
            'Bo',
            'Bp',
            'Bq',
            'Br',
            'Bs',
            'Bt',
            'Bu',
            'Bv',
            'Bw',
            'Bx',
            'By',
            'Bz',
            'I',
            'MSDW_NOT APPLICABLE',
            'MSDW_OTHER',
            'MSDW_UNKNOWN',
            'NOT AVAILABLE',
            'Non Hispanic',
            'O',
            'Other',
            'Pk',
            'Pl',
            'Pm',
            'Po',
            'Ps',
            'Pv',
            'U',
            'Unk',
            'Unknown',
            'W'
        ],
        RaceType.WHITE: [
            'Caucasian (White)',
            'White'
        ]
    }
    
    # value mapping
    data['race'] = (
        data.race.map({
            label: cat for cat, labels in RACE_MAPPING.items()
            for label in labels
        }).astype(pd.api.types.CategoricalDtype(RaceType))
    )
    
    # pre-encoding nan handling
    for column in ['gender', 'race']:
        enc = OneHotEncoder(sparse=False)
        transformed_data = enc.fit_transform(data[[column]])
        transformed_columns = pd.DataFrame(
            data=transformed_data, 
            columns=[f'{column}_{c}' for c in enc.categories_[0]]
        )
        data = data.join(transformed_columns)
        del data[column]
        
    label_encoded_data = data.copy()
    
    scaled_data, _ = Scale().execute(data=label_encoded_data, target=target)
    
    imputed_data, _ = Impute().execute(data=scaled_data, imputation_method=morpher.config.imputers.DEFAULT)
    
    return imputed_data

data_loader = DataLoader(agg_func_regex, prepare_data_function)

In [9]:
from itertools import product

In [10]:
thresholds_numeric = [(x / 100) for x in range(5, 100, 5)]
observation_windows_numeric = [[x, -1] for x in range(-361, -1, 30)]
thresholds_occurring = [(x / 100) for x in range(5, 100, 5)]
observation_windows_occurring = [[x, -1] for x in range(-361, -1, 30)]

In [11]:
import morpher.config

In [12]:
iterator = product(
    thresholds_numeric,
    observation_windows_numeric,
    thresholds_occurring,
    observation_windows_occurring
)

configs = [x for x in iterator if x[0] == x[2] and x[1] == x[3]]
targets = ['urinary_tract_infection_onset_from_0_days_after_to_365_days_after']
algorithms = [morpher.config.algorithms.GBDT, morpher.config.algorithms.RF, morpher.config.algorithms.DT, morpher.config.algorithms.LR]
samplers = samplers = [morpher.config.samplers.RANDOM, morpher.config.samplers.URANDOM, morpher.config.samplers.SMOTE, morpher.config.samplers.NOSAMPLER]

In [14]:
pipeline = training.execute(
    comment='baseline-2 UTI no RFE',
    version='12.0.0',
    cohort=cohort,
    onset_dataframe=onset_dataframe,
    feature_pipeline=feature_pipeline,
    data_loader=data_loader,
    observation_iterator=configs,
    targets=targets,
    algorithms=algorithms,
    samplers=samplers,
    feature_type_occurring="occurring",
    feature_type_numeric="numeric",
    rfe__run=False,
    rfe__step_size=50
)

Columns (11,18,19,20,24,35,64,83,90,92,94,96,97,106,114,117,120,125,126,133,136,140,141,142,143,144,150,178,235,244,248,288,311,317,320,325,326,330,334,341,353,359,360,361,365,402,421,425,430,434,436,439,446) have mixed types. Specify dtype option on import or set low_memory=False.


Fetching data for <fiber.condition.mrns.MRNs object at 0x7ff9aaf620b8>
Fetching data for Patient (...)
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 527.872ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 504.324ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 440.623ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 926.994ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 860.202ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 836.471ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 823.819ms

*** Training of model 'GradientBoostingClassifier' started.
*** 

lbfgs failed to converge. Increase the number of iterations.


*** Training of classifier ready. Time elapsed: 165.65ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 187.431ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 200.777ms

*** Training of model 'LogisticRegression' started.


lbfgs failed to converge. Increase the number of iterations.


...

Fetching data for <fiber.condition.mrns.MRNs object at 0x7ff9aabd6fd0>
Fetching data for Patient (...)
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 288.879ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 293.328ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 300.868ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 301.87ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 307.932ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 186.822ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 178.458ms

*** Training of model 'GradientBoostingClassifier' started.
*** T

*** Training of classifier ready. Time elapsed: 71.128ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 60.246ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 41.847ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 49.557ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 40.038ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 39.349ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 43.226ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 33.676ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 41.274ms

*** Training of model 'LogisticRegression' started.
*** Traini

*** Training of classifier ready. Time elapsed: 36.461ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 10.589ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 19.473ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 12.774ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 37.934ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 25.255ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 21.075ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 27.527ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 44.79ms

*** Training of model 'LogisticRegression' started.
*** Trainin

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 4.892ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 5.572ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 4.302ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 21.603ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 19.414ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 19.639ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 25.191ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 19.584ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 12.195ms

*

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 3.772ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 3.874ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 3.998ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 3.663ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 3.404ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 4.002ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 3.979ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 3.934ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time e

In [15]:
pipeline.id

238